In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler


In [6]:
creditcard_data = pd.read_csv('C:/Users/bam/Documents/Data/creditcard.csv')
fraud_data = pd.read_csv('C:/Users/bam/Documents/Data/Fraud_Data.csv')

# Feature and target separation
X_credit = creditcard_data.drop(columns=['Class'])
y_credit = creditcard_data['Class']

X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']

# Train-test split
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)


In [7]:
model = LogisticRegression()
model.fit(X_train_credit, y_train_credit)
predictions = model.predict(X_test_credit)

# Evaluation
print(classification_report(y_test_credit, predictions))
print("ROC AUC Score:", roc_auc_score(y_test_credit, predictions))


c:\Users\bam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.61      0.56      0.59        98

    accuracy                           1.00     56962
   macro avg       0.81      0.78      0.79     56962
weighted avg       1.00      1.00      1.00     56962

ROC AUC Score: 0.7803044930690339
